In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.externals import joblib
import re
from sklearn import metrics as m
from IPython.display import clear_output
import timeit

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pd.set_option('display.max_columns', None)

In [6]:
# train = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/dataset/UNSW-NB15_full_training_set.csv')
# test = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/dataset/UNSW-NB15_full_testing_set.csv')
# real = pd.concat([train, test], ignore_index=True)
features = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/dataset/features.csv')
real = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/dataset/UNSW-NB15_1.csv', names=features)
real = real.head(10000)
real.rename(index=str, inplace=True, columns={
    'srcip': 'ipv4_src',
    'sport': 'port_src',
    'dstip': 'ipv4_dst',
    'dsport': 'port_dst'})
real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)

/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
real.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [8]:
real.shape

(10000, 49)

In [0]:
df = real.loc[:'3015']
pkts_count = df[['Spkts','Dpkts']].sum()
pkts_count.sum()

## Flows evaluation

In [0]:
input_file = '22-01-2015_1_50000_pcap0_w30'

In [0]:
flows = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/dumps/' + input_file + '.dump', delimiter=',')
flows['real_label'] = 0

In [15]:
flows.shape

(1482, 15)

In [16]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

28879

In [17]:
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,175.45.176.0,13284,149.171.126.16,80,6,1558,352,14,6,4,7,0,Normal,49.91%,0
1,59.166.0.5,3593,149.171.126.5,53,17,160,192,2,2,17,63,0,Normal,49.91%,0
2,59.166.0.3,49664,149.171.126.0,53,17,174,206,2,2,27,80,0,Normal,49.91%,0
3,59.166.0.5,6645,149.171.126.2,80,6,272,140,4,2,18,65,0,Normal,0.00%,0
4,59.166.0.3,42587,149.171.126.8,25,6,404,444,6,4,4,55,0,Normal,1.60%,0


In [18]:
flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)
start = timeit.default_timer()
for i in flows.index:
  clear_output(wait=True)
  
  tmp = real['Label'].loc[((real['ipv4_src'] == flows['ipv4_src'][i]) & 
               (real['port_src'] == flows['port_src'][i]) &
               (real['ipv4_dst'] == flows['ipv4_dst'][i]) &
               (real['port_dst'] == flows['port_dst'][i])) | (
               (real['ipv4_src'] == flows['ipv4_dst'][i]) & 
               (real['port_src'] == flows['port_dst'][i]) &
               (real['ipv4_dst'] == flows['ipv4_src'][i]) &
               (real['port_dst'] == flows['port_src'][i]))]
  flows['real_label'][i] = tmp.values[0]
  
  stop = timeit.default_timer()
  if (float(float(i + 1) / float(len(flows.index)) * 100) < 5):
    expected_time = 'Calculating...'
  else:
    time_perc = timeit.default_timer()
    expected_time = np.round((float(time_perc - start) / (float(i + 1) / float(len(flows.index)))) / 60,2)
  print 'Progress: ' + str(i+1) + '/' + str(len(flows.index))
  print 'Percentage: ' + str(np.round(float(float(i + 1) / float(len(flows.index))) * 100, 2)) + '%'
  print 'Run time: ' + str(np.round((stop - start) / 60, 2)) + ' minutes'
  print 'Expected run time: ' + str(expected_time) + ' minutes'

Progress: 1482/1482
Percentage: 100.0%
Run time: 2.74 minutes
Expected run time: 2.74 minutes


In [19]:
custom = flows.replace(to_replace='Warning', value='Attack')
custom = custom.replace(to_replace='Best_effort', value='Attack')
evaluation = custom[['ipv4_src', 'category', 'real_label']]
evaluation['ipv4_src'] = evaluation.apply(lambda row: 'Malicious user' if row['real_label'] == 1 else 'Normal user', axis=1)
evaluation = evaluation[['ipv4_src', 'category']]
evaluation['count'] = 1
evaluation = evaluation.groupby(['ipv4_src', 'category']).count()
evaluation

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


count
ipv4_src       category       
Malicious user Attack       19
               Normal       12
Normal user    Attack       12
               Normal     1439

In [20]:
e = evaluation.unstack(fill_value=0)
e
try:
    TP = e.loc['Malicious user'].loc['count'].loc['Attack']
except:
    TP = 0
try:
    TN = e.loc['Normal user'].loc['count'].loc['Normal']
except:
    TN = 0
try:
    FP = e.loc['Normal user'].loc['count'].loc['Attack']
except:
    FP = 0
try:
    FN = e.loc['Malicious user'].loc['count'].loc['Normal']
except:
    FN = 0
print 'TP:', TP
print 'TN:', TN
print 'FP:', FP
print 'FN:', FN
try:
    acc = float(TP + TN)/float(TP + TN + FP + FN)
except:
    acc = 0
print ('Accuracy is %.2f%%' % (acc*100))
try:
    sen = float(TP)/float(TP + FN)
except:
    sen = 0
print ('Sensitivity is %.2f%%' % (sen*100))
try:
    spe = float(TN)/float(TN + FP)
except:
    spe = 0
print ('Specificity is %.2f%%' % (spe*100))
try:
    pre = float(TP)/float(TP + FP)
except:
    pre = 0
print ('Precision is %.2f%%' % (pre*100))
try:
    f1_score = 2*(float(pre*sen)/float(pre+sen))
except:
    f1_score = 0
print ('F1 score is %.2f%%' % (f1_score*100))

TP: 19
TN: 1439
FP: 12
FN: 12
Accuracy is 98.38%
Sensitivity is 61.29%
Specificity is 99.17%
Precision is 61.29%
F1 score is 61.29%


In [0]:
flows.to_csv(path_or_buf='/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/processed_csv/' + input_file + '.csv', index=False)

In [0]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

49513

# Comparision

In [0]:
means_names = ['Name','Accuracy','Sensitivity','Specificity','Precision','F1-score','Processed packets']
overall = pd.DataFrame(columns=means_names)

In [22]:
overall = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/processed_csv/overall_stats.csv')
# overall = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/processed_csv/overall_stats_small_pcaps.csv')
overall

,Name,Accuracy,Sensitivity,Specificity,Precision,F1-score,Processed packets
0,22-01-2015_1_100000_pcap0_w5,0.988048,0.675676,0.993912,0.675676,0.675676,84206
1,22-01-2015_1_100000_pcap0_w10,0.993253,0.658537,1.000000,1.000000,0.794118,96164
2,22-01-2015_1_100000_pcap0_w20,0.985915,0.666667,0.992563,0.651163,0.658824,99432
3,22-01-2015_1_100000_pcap0_w30,0.981760,0.512195,0.992320,0.600000,0.552632,86874
4,22-01-2015_1_200000_pcap0_w5,0.990997,0.633803,1.000000,1.000000,0.775862,177996
5,22-01-2015_1_200000_pcap0_w10,0.987205,0.638889,0.995632,0.779661,0.702290,187193
6,22-01-2015_1_200000_pcap0_w20,0.988669,0.578947,1.000000,1.000000,0.733333,176822
7,22-01-2015_1_200000_pcap0_w30,0.987813,0.539474,1.000000,1.000000,0.700855,184857
8,22-01-2015_1_500000_pcap0_w5,0.978738,0.398058,0.999766,0.984000,0.566820,745063
9,22-01-2015_1_500000_pcap0_w10,0.980610,0.380623,0.999889,0.990991,0.550000,714745


In [0]:
means_names = ['Name','Accuracy','Sensitivity','Specificity','Precision','F1-score','Processed packets']
df = pd.DataFrame([[input_file, acc, sen, spe, pre, f1_score, packets_count.sum()]], columns=means_names)
overall = overall.append(df)

In [24]:
overall

,Name,Accuracy,Sensitivity,Specificity,Precision,F1-score,Processed packets
0,22-01-2015_1_100000_pcap0_w5,0.988048,0.675676,0.993912,0.675676,0.675676,84206
1,22-01-2015_1_100000_pcap0_w10,0.993253,0.658537,1.000000,1.000000,0.794118,96164
2,22-01-2015_1_100000_pcap0_w20,0.985915,0.666667,0.992563,0.651163,0.658824,99432
3,22-01-2015_1_100000_pcap0_w30,0.981760,0.512195,0.992320,0.600000,0.552632,86874
4,22-01-2015_1_200000_pcap0_w5,0.990997,0.633803,1.000000,1.000000,0.775862,177996
5,22-01-2015_1_200000_pcap0_w10,0.987205,0.638889,0.995632,0.779661,0.702290,187193
6,22-01-2015_1_200000_pcap0_w20,0.988669,0.578947,1.000000,1.000000,0.733333,176822
7,22-01-2015_1_200000_pcap0_w30,0.987813,0.539474,1.000000,1.000000,0.700855,184857
8,22-01-2015_1_500000_pcap0_w5,0.978738,0.398058,0.999766,0.984000,0.566820,745063
9,22-01-2015_1_500000_pcap0_w10,0.980610,0.380623,0.999889,0.990991,0.550000,714745


In [26]:
overall['Accuracy'].mean()

0.9847141055814075

In [27]:
overall['Sensitivity'].mean()

0.5943280718233381

In [28]:
overall['Specificity'].mean()

0.9952734527043356

In [29]:
overall['Precision'].mean()

0.806577063117818

In [30]:
overall['F1-score'].mean()

0.663939747471305

In [0]:
overall.to_csv(path_or_buf='/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/processed_csv/overall_stats.csv', index=False)

## Notes and unused commands

In [0]:
input_file = '22-01-2015_1_500000_pcap0_w30'
flows = pd.read_csv('/content/drive/My Drive/FIIT STU/Ing/4. semester/Diplomový projekt 3/results/processed_csv/'+ input_file +'.csv')
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,175.45.176.0,13284,149.171.126.16,80,6,1558,352,14,6,8,17,0,Normal,49.55%,1
1,59.166.0.5,3593,149.171.126.5,53,17,160,192,2,2,68,391,0,Normal,49.91%,0
2,59.166.0.3,49664,149.171.126.0,53,17,174,206,2,2,77,415,0,Normal,49.91%,0
3,59.166.0.5,6645,149.171.126.2,80,6,1164,964,8,10,48,455,0,Normal,49.91%,0
4,59.166.0.3,42587,149.171.126.8,25,6,38086,3968,52,42,13,390,0,Normal,49.91%,0
